In [ ]:
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
num_of_nodes = 1000
num_of_edges = 5
k_nearest_neighbors = 6
rewiring_probability = 0.1
edge_creation_probability = 0.1

ba_graph = nx.barabasi_albert_graph(num_of_nodes, num_of_edges)
ws_graph = nx.watts_strogatz_graph(num_of_nodes, k_nearest_neighbors, rewiring_probability)
er_graph = nx.erdos_renyi_graph(num_of_nodes, edge_creation_probability)

In [ ]:
# Disease Parameters
betas = [0.05, 0.1, 0.3]
gammas = [0.005, 0.1, 0.5]

results = {}

for beta in betas:
    for gamma in gammas:
        config = mc.Configuration()
        config.add_model_parameter('beta', beta)
        config.add_model_parameter('gamma', gamma)
        config.add_model_parameter("percentage_infected", 0.1) # 10% initially infected

        for graph, name in [(ba_graph, "Barabási-Albert"), (ws_graph, "Watts-Strogatz"), (er_graph, "Erdős-Rényi")]:
            model = ep.SIRModel(graph)
            model.set_initial_status(config)
            iterations = model.iteration_bunch(200)
            
            # Store results
            key = (name, beta, gamma)
            results[key] = iterations

In [ ]:
def show_graph(network_name, beta, gamma, time_step):
    if network_name == "Barabási-Albert":
        network = ba_graph
    elif network_name == "Watts-Strogatz":
        network = ws_graph
    elif network_name == "Erdős-Rényi":
        network = er_graph

    key = (network_name, beta, gamma)
    if key not in results:
        print(f"No results for {key}")
        return

    current_status = results[(key)][time_step]['status']
    colors = [ "yellow" if current_status.get(node) == 0 
               else "red" if current_status.get(node) == 1 
               else "green" for node in network.nodes() ]

    clear_output(wait=True)
    nx.draw(network, node_color=colors, with_labels=False)
    plt.title(f"{network_name} at time {time_step} with Beta={beta} and Gamma={gamma}")
    plt.show()

network_dropdown = widgets.Dropdown(
    options=["Barabási-Albert", "Watts-Strogatz", "Erdős-Rényi"],
    value="Watts-Strogatz",
    description='Network:',
)

beta_slider = widgets.SelectionSlider(
    options=betas,
    value=betas[0],
    description='Beta:',
    continuous_update=False
)

gamma_slider = widgets.SelectionSlider(
    options=gammas,
    value=gammas[0],
    description='Gamma:',
    continuous_update=False
)

time_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(iterations)-1,
    step=1,
    description='Time Step:',
    continuous_update=False
)

widgets.interactive(show_graph, 
                    network_name=network_dropdown, 
                    beta=beta_slider, 
                    gamma=gamma_slider, 
                    time_step=time_slider)